In [1]:
from transformers import pipeline
import pandas as pd
from sklearn.metrics import f1_score, accuracy_score

In [2]:
pipe = pipeline(model='fzanartu/flicc', device='cuda')

In [3]:
datasets = {'cherry picking':'https://raw.githubusercontent.com/Tariq60/fallacy-detection/master/data/climate/test/Cherry%20Picking.tsv',
            'single cause': 'https://raw.githubusercontent.com/Tariq60/fallacy-detection/master/data/climate/test/False%20Cause.tsv',
            'fake experts': 'https://raw.githubusercontent.com/Tariq60/fallacy-detection/master/data/climate/test/False%20Authority.tsv'
            }

for label, link in datasets.items():

    data = pd.read_csv(link, sep='\t')
    data['label'] = label
    al_true = data['label'].to_list()
    al_results = pipe(data['fact_checked_segment'].to_list())
    al_pred = [x.get('label') for x in al_results]
    print(f'label: {label}, acc: {round(accuracy_score(al_true, al_pred),2)}, f1: {round(f1_score(al_true, al_pred, average="weighted", zero_division=0.0),2)}')


label: cherry picking, acc: 0.48, f1: 0.65
label: single cause, acc: 0.22, f1: 0.36
label: fake experts, acc: 0.1, f1: 0.18


In [4]:
jin = pd.read_csv('https://raw.githubusercontent.com/causalNLP/logical-fallacy/main/data/climate_test.csv', usecols=['source_article', 'logical_fallacies'])
jin = jin[jin['logical_fallacies'].isin(['intentional', 'ad hominem', 'false dilemma'])]
jin = jin.dropna().reset_index(drop=True)

fallacies_map = {'intentional':'cherry picking', 
                 'ad hominem':'ad hominem',
                 'false dilemma':'false choice'}


for fallacy in jin['logical_fallacies'].unique():
    fallacies = jin[jin['logical_fallacies'] == fallacy]['source_article'].to_list()
    jin_true = [fallacies_map.get(fallacy)] * len(fallacies)
    jin_results = pipe(fallacies)
    jin_pred = [x.get('label') for x in jin_results]
    print(f'label: {fallacies_map.get(fallacy)}, acc: {round(accuracy_score(jin_true, jin_pred),2)}, f1: {round(f1_score(jin_true, jin_pred, average="weighted", zero_division=0.0),2)}')

label: cherry picking, acc: 0.22, f1: 0.36
label: ad hominem, acc: 0.69, f1: 0.81
label: false choice, acc: 0.5, f1: 0.67
